# Rank

Carreguem les variables i els diccionaries de carectristiques de validadcio i entrenament

In [4]:
import os
import pickle
import numpy as np
from get_params import get_params
from sklearn.metrics.pairwise import pairwise_distances

def rank(params):

    
    val_features = pickle.load(open(os.path.join(params['root'],params['root_save'],params['feats_dir'],
                             params['split'] + "_" + str(params['descriptor_size']) + "_"
                             + params['descriptor_type'] + "_" + params['keypoint_type'] + '.p'),'rb'))

    train_features = pickle.load(open(os.path.join(params['root'],params['root_save'],params['feats_dir'],
                             'train' + "_" + str(params['descriptor_size']) + "_"
                             + params['descriptor_type'] + "_" + params['keypoint_type'] + '.p'),'rb'))



ImportError: No module named get_params

Per a cada imatge obtenim, les seves característiques i les posem en una matriu per crear una llista que dira rankings, seguidament guardem el fitxer de text
 

In [3]:

    for val_id in val_features.keys():

       
        bow_feats = val_features[val_id]

        
        ranking = train_features.keys()

        X = np.array(train_features.values())

        # The .squeeze() method reduces the dimensions of an array to the minimum. E.g. if we have a numpy array of shape (400,1,100) it will transform it to (400,100)
        distances = pairwise_distances(bow_feats,X.squeeze())


        # Sort the ranking according to the distances. We convert 'ranking' to numpy.array to sort it, and then back to list (although we could leave it as numpy array).
        ranking = list(np.array(ranking)[np.argsort(distances.squeeze())])

       
        outfile = open(os.path.join(params['root'],params['root_save'],params['rankings_dir'],params['descriptor_type'],params['split'],val_id.split('.')[0] + '.txt'),'w')

        for item in ranking:

            outfile.write(item.split('.')[0] + '\n')

        outfile.close()

if __name__ == "__main__":

    params = get_params()
    rank(params)

IndentationError: unexpected indent (<ipython-input-3-af0a62476953>, line 2)

# Eval_Rankings

In [ ]:
def display(params,query_id,ranking,relnotrel):

    ''' Display the first elements of the ranking '''


   
    query_im =  cv2.imread(os.path.join(params['root'],params['database'],params['split'], 'images',query_id.split('.')[0] + '.jpg'))

    
    if query_im is None:
        query_im =  cv2.imread(os.path.join(params['root'],params['database'],params['split'], 'images',query_id.split('.')[0] + '.JPG'))



    query_im = cv2.cvtColor(query_im,cv2.COLOR_BGR2RGB)
    query_im = cv2.copyMakeBorder(query_im,100,100,100,100,cv2.BORDER_CONSTANT,value=[0,0,255])
   
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(4, 4, 1)

    
    ax.imshow(query_im)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

Obtenim les anotacions correctes

In [ ]:
def read_annotation(params):

    annotation_val = pd.read_csv(os.path.join(params['root'],params['database'],params['split'],'annotation.txt'), sep='\t', header = 0)
    annotation_train = pd.read_csv(os.path.join(params['root'],params['database'],'train','annotation.txt'), sep='\t', header = 0)

    return annotation_val,annotation_train

Obtenim cada clase de les anotacions d'entrenament si coincideixen vol dir que son correctes retornem un 1 
si no retornem un 0

In [ ]:
def get_hitandmiss(ranking,query_class,annotation_train):

  
    relnotrel = []

   
    for i in ranking[0].tolist():

       
        i_class = list(annotation_train.loc[annotation_train['ImageID'] == i]['ClassID'])[0]

       
        if query_class == i_class:

            
            relnotrel.append(1)
        else:

            
            relnotrel.append(0)

    return relnotrel

Calculem la precisio

In [ ]:
def AveragePrecision(relist):
    '''Takes a hit & miss list with ones and zeros and computes its average precision'''

    
    accu = 0

   
    numRel = 0

    # For all elements in the hit & miss list
    for k in range(len(relist)):

        # If the value is 1
        if relist[k] == 1:

            # We add 1 to the number of correct instances
            numRel = numRel + 1

            # We calculate the precision at k (+1 because we start at 0) and we accumulate it
            accu += float( numRel )/ float(k+1)

    # When we finish, we divide by the total number of relevant instances, which is the sum of ones in the list
    return (accu/np.sum(relist))


Carreguem les clases de validacio les quals seran evaluades

In [ ]:
def load_ranking(params,query_id, annotation_val):

    ''' Loads and  returns the ranking from the txt. Returns the true class of the query image as well.'''

    # Get the true class of the validation image for which we will evaluate the ranking
    query_class = list(annotation_val.loc[annotation_val['ImageID'] == query_id.split('.')[0]]['ClassID'])[0]

    # Open its ranking file
    ranking = pd.read_csv(os.path.join(params['root'],params['root_save'],params['rankings_dir'],params['descriptor_type'],params['split'],query_id.split('.')[0] + '.txt'),header= None)

    return query_class, ranking


convertim el elements a un string i les llistem en el ranking

In [ ]:
def save_ranking_file(file_to_save,image_id,ranking):

    '''
    :param file_to_save: name of the file to be saved
    :param image_id: name of the query image
    :param ranking: ranking for the image image_id
    :return: the updated state of the file to be saved
    '''

    # Write query name
    file_to_save.write(image_id.split('.')[0] + ',')


    ranking = np.array(ranking).astype('str').tolist()

    
    for item in ranking:
        file_to_save.write(item[0] + " ")

    file_to_save.write('\n')

    return file_to_save

Evaluem les llistes creades anteriorment i calculem la acuracitat per a cada edifici i els imprimim. Activant una linia podem imprimir els resultats amb les imatges recuadrades amb verd si ha accertat o en vermell si ha fallat

In [ ]:
def eval_rankings(params):

    ap_list = []

    
    if params['save_for_kaggle']:

        file_to_save = open(os.path.join(params['root'],params['root_save'],params['kaggle_dir'],params['descriptor_type'] + '_' + params['split'] + '_ranking.csv'),'w')

      
        file_to_save.write("Query,RetrievedDocuments\n")

    # Create a dictionary to store the accumulated AP for each class
    dict_ = {key: 0 for key in params['possible_labels']}

    # Get true annotations
    annotation_val, annotation_train = read_annotation(params)
    '''  
    # Used once to save ranking annotations for kaggle competition.
    gt_file_to_save = open(os.path.join(params['root'],params['root_save'],params['kaggle_dir'],params['split'] + '_rankingannotation.csv'),'w')
    kaggle_scripts.convert_ranking_annotation(annotation_val,annotation_train,gt_file_to_save)
  '''  

    # For all generated rankings
    for val_id in os.listdir(os.path.join(params['root'],params['root_save'],params['rankings_dir'],params['descriptor_type'],params['split'])):

        query_class, ranking = load_ranking(params,val_id,annotation_val)

       
        if not query_class == "desconegut":
            #single_eval(params,val_id)  # <-----
            if params['save_for_kaggle']:

                file_to_save = save_ranking_file(file_to_save,val_id,ranking)

            
            relnotrel = get_hitandmiss(ranking,query_class,annotation_train)

           
            ap = AveragePrecision(relnotrel)


            # OPTIONAL: Add the AP to the according dictionary entry
            dict_[query_class] += ap

            # Store it
            ap_list.append(ap)


    if params['save_for_kaggle']:

        file_to_save.close()

    return ap_list, dict_


In [ ]:
def single_eval(params,query_id):

    # We get the true annotations of both sets. We do this in order to display this information as well.
    annotation_val, annotation_train = read_annotation(params)

    # We get the ranking and the true class of the query
    query_class, ranking = load_ranking(params,query_id,annotation_val)

    # I made sure I was not picking an image from the "desconegut" class ...
    print query_class

    # The ranking is composed of the training images. It should be of size 450.
    print len(ranking)

    # We get the hit/miss list for the ranking
    relnotrel = get_hitandmiss(ranking,query_class,annotation_train)

    display(params,query_id,ranking,relnotrel)

if __name__ == "__main__":

    params = get_params()

    ap_list, dict_ = eval_rankings(params)
    print 'Accuracy:'
    print 'Mean:',np.mean(ap_list)

    for id in dict_.keys():
        if not id == 'desconegut':
            # We divide by 10 because it's the number of images per class in the validation set.
            print id+':', dict_[id]/10